<h2> Unsupervised Pretraining </h2> 

When we do not have much labeled data, but we cannot find a model trained on a similar task. 

If the data you have is unlabelled, you may still be able to perform unsupervised pretraining on it. 

If you can gather plenty of unlabeled training data, you can try to train the layers one by one starting with the lowest layer and going up. 

<h4> Boltzmaan Machine </h4> 

Instead of using a deterministic step function to decide which value to output, these neurons output 1 with some probability, and 0 otherwise.

If we keep a Boltzmaan machine running long enough, the probbility of observing a perticular outcome will only be a function of the connection weights and bias terms, not of the original configuration.

Training a Boltzmann machine means finding the parameters which make the network approximate the training sets probability distribution.

The point where your model no longer depends on its intital state, it has reached thermal equilibrium where there is no thermal gradient to cause any change. This is a generative model.

<h4> Restricted Boltzmaan Machines </h4> 

RBMs are shallow neural networks with only 2 layers and is an unsupervised training method used to find patterns in data by reconstructing the input. It is called restricted because neurons in the same layer are not connected. 

RBMs can be used for both:
- reconstruction of input
- classify instances lacking classification by using the concept that similar neurons will fire in similar inputs. 

Given that you have more unlabelled data, you start off by building your network using only unlabelled data such that your input in the forward pass with be equal to the backward output. Then after training sufficient hidden layers, you feed in labelled data to train the output layer. 


Currently available methods for us to speed up training are:
- Initialization strategy
- Good activation function
- Batch Normalization
- Reusing Pretrained network

<h2> Faster Optimizers </h2> 

<h2> Momentum Optimization </h2> 

It starts out slow and starts increasing speed as it picks up momentum, as it goes down the gradient slope. 

In gradient descent, it simply subtracts the existing theta values by the current gradient. It has no idea of the earlier gradient of the previous step. 

Momentum optimization subsctracts the local gradient from the momentum vector(multiplied by m) and it updates the weights by simply adding this momentum vector. 

The gradient vector thats multiplied with the learning rate is used to control the accelerating here. not the speed. Hence a larger negative gradient results in a higher accelaration. 


Momentum optimization can update weights 10 times faster for a typical beta value of 0.9. This means it will not be stuck in plateaus when reaching global minima, hence it can work with non normalized data faster too. 

momentum <= friction * momentum - lr * gradient
weights <= weights + momentum 

Notice we have added a  new parameter to tune in momentum